## Knowledge Retrieval

Retrieval augments the Assistant with knowledge from outside its model, such as proprietary product information or documents provided by your users. Once a file is uploaded and passed to the Assistant, OpenAI will automatically chunk your documents, index and store the embeddings, and implement vector search to retrieve relevant content to answer user queries.
The model then decides when to retrieve content based on the user Messages. The Assistants API automatically chooses between two retrieval techniques:

* it either passes the file content in the prompt for short documents, or
* performs a vector search for longer documents





Code Interpreter can parse data from files. This is useful when you want to provide a large volume of data to the Assistant or allow your users to upload their own files for analysis. Note that files uploaded for Code Interpreter are not indexed for retrieval

Files have a maximum size of 512 MB. Code Interpreter supports a variety of file formats including .csv, .pdf, .json and many more.

In [42]:
from openai import OpenAI
import json
from dotenv import load_dotenv, find_dotenv

_ : bool = load_dotenv(find_dotenv()) # read local .env file

In [43]:
client : OpenAI = OpenAI()

In [65]:
from openai.types.beta import Assistant

# Upload a file with an "assistants" purpose
file = client.files.create(
  file=open("zia_profile.pdf", "rb"),
  purpose='assistants'
)

print(file)

FileObject(id='file-lGpQ0KKL54gLrmYxz4SFova1', bytes=48802, created_at=1702327528, filename='zia_profile.pdf', object='file', purpose='assistants', status='processed', status_details=None)


In [66]:
assistant: Assistant = client.beta.assistants.create(
  name="Student Support Assistant",
  instructions="You are a student support chatbot. Use your knowledge base to best respond to student queries about Zia U. Khan.",
  model="gpt-3.5-turbo-1106",
  tools=[{"type": "retrieval"}],
  file_ids=[file.id]
)

In [67]:
from openai.types.beta.thread import Thread

thread: Thread  = client.beta.threads.create()

print(thread)

Thread(id='thread_DGNpJ3BQDI0aWMUpwoUEcxXK', created_at=1702327537, metadata={}, object='thread')


In [68]:
from openai.types.beta.threads.thread_message import ThreadMessage

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="When and which city Zia U. Khan was born and from where he completed his matriculation"
)

In [72]:
from openai.types.beta.threads.run import Run

run: Run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Pakistani. The user is the student of PIAIC."
)


* By default, a Run will use the model and tools configuration specified in Assistant object, but you can override most of these when creating the Run for added flexibility
* Note: file_ids associated with the Assistant cannot be overridden during Run creation. You must use the modify Assistant endpoint to do this.


In [73]:
run: Run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

print(run)

Run(id='run_9QVOqkGBJg1LLLG6nKQWBY8m', assistant_id='asst_F1DAOeElMuP32n0Kt4GJNWiA', cancelled_at=None, completed_at=1702327574, created_at=1702327573, expires_at=None, failed_at=None, file_ids=['file-lGpQ0KKL54gLrmYxz4SFova1'], instructions='Please address the user as Pakistani. The user is the student of PIAIC.', last_error=None, metadata={}, model='gpt-3.5-turbo-1106', object='thread.run', required_action=None, started_at=1702327573, status='completed', thread_id='thread_DGNpJ3BQDI0aWMUpwoUEcxXK', tools=[ToolAssistantToolsRetrieval(type='retrieval')])


### Polling for updates
In order to keep the status of your run up to date, you will have to periodically retrieve the Run object. You can check the status of the run each time you retrieve the object to determine what your application should do next. We plan to add support for streaming to make this simpler in the near future.

Thread locks

In [74]:
# from openai.resources.beta.threads.messages.messages import SyncCursorPage 

messages: list[ThreadMessage] = client.beta.threads.messages.list(
  thread_id=thread.id
)

for m in reversed(messages.data):
  print(m.role + ": " + m.content[0].text.value)

user: When and which city Zia U. Khan was born and from where he completed his matriculation
assistant: I will look into the file to find the information about Zia U. Khan's birth city and the location of his matriculation.
assistant: Zia U. Khan was born in an army garrison in Sialkot in 1961【7†source】. Additionally, he completed his matriculation from Habib Public School in Karachi in 1977【7†source】.
